In [1]:
# import libraries
import numpy as np
import pandas as pd

In [2]:
# read the dataframe
df = pd.read_csv('../data/Addis_Ababa_Solcast.csv')

In [3]:
df.sample(5)

,PeriodEnd,PeriodStart,Period,AirTemp,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m
20440,2021-04-30T18:00:00Z,2021-04-30T17:00:00Z,PT60M,14.2,87.0,0,0,0,26.5,86.7,764.7,138,1.6
22340,2021-07-18T22:00:00Z,2021-07-18T21:00:00Z,PT60M,12.5,8.7,0,0,0,24.0,90.5,765.0,273,1.6
13375,2020-07-10T09:00:00Z,2020-07-10T08:00:00Z,PT60M,18.9,41.0,549,0,549,23.4,61.8,765.3,190,1.7
25283,2021-11-18T13:00:00Z,2021-11-18T12:00:00Z,PT60M,21.6,0.0,91,864,565,11.5,28.5,764.1,105,4.8
7065,2019-10-21T11:00:00Z,2019-10-21T10:00:00Z,PT60M,20.3,9.2,262,683,862,15.4,52.1,764.0,100,5.2


### Changing the names and types of the columns

In [4]:
old_names = ['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi','PrecipitableWater', 'RelativeHumidity',
             'SurfacePressure', 'WindDirection10m', 'WindSpeed10m'] 
new_names = ['PeriodEnd','PeriodStart','Temperature', 'DHI', 'DNI', 'Radiation', 'Precipitation', 'Humidity', 
             'Pressure', 'WindDirection', 'WindSpeed']
df.rename(columns = dict(zip(old_names, new_names)), inplace = True)
df = df.drop(columns=['Period'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26181 entries, 0 to 26180
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PeriodEnd      26181 non-null  object 
 1   PeriodStart    26181 non-null  object 
 2   Temperature    26181 non-null  float64
 3   CloudOpacity   26181 non-null  float64
 4   DHI            26181 non-null  int64  
 5   DNI            26181 non-null  int64  
 6   Radiation      26181 non-null  int64  
 7   Precipitation  26181 non-null  float64
 8   Humidity       26181 non-null  float64
 9   Pressure       26181 non-null  float64
 10  WindDirection  26181 non-null  int64  
 11  WindSpeed      26181 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 2.4+ MB


In [6]:
 # interpret columns as appropriate data types to ensure compatibility
for col in ['Radiation', 'Temperature', 'Pressure', 'Humidity', 'WindDirection', 'WindSpeed']:
    df[col] = df[col].astype(float)

for col in ['PeriodStart', 'PeriodEnd']:
    df[col] = pd.to_datetime(df[col]).dt.to_period('T').dt.to_timestamp()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [7]:
df.head(5)

,PeriodEnd,PeriodStart,Temperature,CloudOpacity,DHI,DNI,Radiation,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31 02:00:00,2018-12-31 01:00:00,9.1,0.0,0,0,0.0,9.1,91.6,782.7,76.0,2.3
1,2018-12-31 03:00:00,2018-12-31 02:00:00,9.0,0.0,0,0,0.0,8.8,92.4,783.0,76.0,2.3
2,2018-12-31 04:00:00,2018-12-31 03:00:00,9.9,2.4,5,17,6.0,8.8,86.8,783.4,78.0,2.5
3,2018-12-31 05:00:00,2018-12-31 04:00:00,11.9,29.2,89,145,127.0,9.1,76.0,784.0,81.0,2.8
4,2018-12-31 06:00:00,2018-12-31 05:00:00,13.9,3.8,85,786,409.0,9.3,66.7,784.5,83.0,3.1


### Aplying correct time zone and winter/summer time

In [8]:
df['PeriodStart'] = df['PeriodStart'].dt.tz_localize('UTC').dt.tz_convert('Africa/Addis_Ababa')
df['PeriodEnd'] = df['PeriodEnd'].dt.tz_localize('UTC').dt.tz_convert('Africa/Addis_Ababa')

### Create sunrise and sunset feature

In [9]:
# Create new features(year, month, day) from start and end time of data collection  
df['Year'] = pd.to_datetime(df['PeriodEnd']).dt.year
df['Month'] = pd.to_datetime(df['PeriodEnd']).dt.month

df['HourPS'] = pd.to_datetime(df['PeriodStart']).dt.hour
df['HourPE'] = pd.to_datetime(df['PeriodEnd']).dt.hour

In [10]:
df.loc[df['Radiation'] == 0]['HourPS'].unique()

array([ 4,  5, 19, 20, 21, 22, 23,  0,  1,  2,  3, 18])

In [11]:
df.loc[df['Radiation'] == 0]['HourPE'].unique()

array([ 5,  6, 20, 21, 22, 23,  0,  1,  2,  3,  4, 19])

In [12]:
# create a new feature for sunrise using the function below; fill time for sunrise
def time_to_sunrise(row):
    """
    A function for identifying time of sunrise.
    """
    value = 0        
    if row['HourPS'] == 4 and row['HourPE'] == 5 and row['Radiation'] > 0:
        value= 5
    elif row['HourPS'] == 5 and row['HourPE'] == 6 and row['Radiation'] > 0:
        value= 6
    return value

# create a new feature for sunset using the function below; fill time for sunset
def time_to_sunset(row):
    """
    A function for identifying time of sunset.
    """
    value= 0
    if row['HourPS'] == 17 and row['HourPE'] == 18 and row['Radiation']> 0:
        value= 18
    elif row['HourPS'] == 18 and row['HourPE'] == 19 and row['Radiation']> 0:
        value= 19
    return value

In [13]:
#Create a new column using the function above
df['Sunrise'] = df.apply(time_to_sunrise, axis=1)

#Create a new column using the function above
df['Sunset'] = df.apply(time_to_sunset, axis=1)

In [14]:
df.Sunrise.unique(), df.Sunset.unique()

(array([0]), array([ 0, 18, 19]))

### Removing night time data

In [15]:
# changing integer values to datetime format
df = df.loc[~((df['Radiation'] == 0) & (df['DNI'] == 0) & (df['DHI'] == 0)),:]

### Grouping by day

In [16]:
df['date'] = pd.to_datetime(df['PeriodEnd']).dt.date

for col in ['Sunrise', 'Sunset']:
    df[col] = df[col].astype(str)
    
agg_dict = {
    'Temperature':'mean',
    'CloudOpacity':'mean',
    'DHI':'mean',
    'DNI':'mean',
    'Radiation':'mean',
    'Precipitation':'mean',
    'Humidity':'mean',
    'Pressure':'mean',
    'WindDirection':'mean',
    'WindSpeed':'mean',
    'Sunrise':'max',
    'Sunset':'max',
}

df_grouped = df.groupby(['date']).agg(agg_dict)
df_grouped.reset_index(inplace=True)
df_grouped

,date,Temperature,CloudOpacity,DHI,DNI,Radiation,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise,Sunset
0,2018-12-31,18.338462,3.800000,76.769231,710.615385,519.384615,10.330769,46.215385,783.176923,81.692308,4.269231,0,19
1,2019-01-01,18.069231,0.284615,54.846154,792.000000,535.769231,9.546154,38.976923,783.884615,87.846154,3.069231,0,19
2,2019-01-02,19.292308,0.738462,57.769231,767.615385,527.153846,14.030769,38.915385,783.661538,90.307692,2.615385,0,19
3,2019-01-03,19.423077,4.692308,67.076923,700.846154,513.692308,17.869231,49.015385,783.930769,77.153846,4.169231,0,19
4,2019-01-04,17.607692,0.438462,49.307692,825.076923,547.461538,6.784615,37.961538,784.176923,71.769231,5.600000,0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,2021-12-21,18.415385,8.184615,130.923077,565.461538,482.692308,12.484615,50.684615,766.307692,103.076923,4.815385,0,19
1087,2021-12-22,17.976923,11.784615,153.000000,509.923077,470.692308,12.530769,50.184615,765.784615,104.461538,4.307692,0,19
1088,2021-12-23,18.538462,2.915385,70.769231,707.846154,513.615385,14.223077,47.569231,764.830769,106.384615,4.269231,0,19
1089,2021-12-24,18.892308,24.146154,212.538462,341.846154,426.000000,11.484615,34.430769,764.638462,122.076923,2.769231,0,19


In [17]:
df_grouped['Sunrise'] = df_grouped['Sunrise'].astype(int) 
df_grouped['Sunset'] = df_grouped['Sunset'].astype(int) 
df_grouped['Month'] = pd.to_datetime(df_grouped['date']).dt.month

In [18]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091 entries, 0 to 1090
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1091 non-null   object 
 1   Temperature    1091 non-null   float64
 2   CloudOpacity   1091 non-null   float64
 3   DHI            1091 non-null   float64
 4   DNI            1091 non-null   float64
 5   Radiation      1091 non-null   float64
 6   Precipitation  1091 non-null   float64
 7   Humidity       1091 non-null   float64
 8   Pressure       1091 non-null   float64
 9   WindDirection  1091 non-null   float64
 10  WindSpeed      1091 non-null   float64
 11  Sunrise        1091 non-null   int64  
 12  Sunset         1091 non-null   int64  
 13  Month          1091 non-null   int64  
dtypes: float64(10), int64(3), object(1)
memory usage: 119.5+ KB


In [19]:
df_grouped.shape

(1091, 14)

In [20]:
df_grouped.to_csv('../data/Addis_Ababa_Solcast_clean.csv', index=False)